In [1]:
!pip install datasets
import datasets
from datasets import load_dataset
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


In [2]:
!pip install transformers torch datasets seqeval
from transformers import BertTokenizer, BertForTokenClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import ast
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cach

In [3]:
naamapadam = load_dataset("ai4bharat/naamapadam", 'hi') #naamapadam
hiner = load_dataset('cfilt/HiNER-original') #HiNER


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/985787 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/867 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13460 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/75827 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10851 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21657 [00:00<?, ? examples/s]

In [4]:
# Convert to pandas DataFrame
hiner_train_df = pd.DataFrame(hiner['train'].remove_columns('id'))
naamapadam_train_df = pd.DataFrame(naamapadam['train'])


In [5]:
# Combine the datasets
combined_ner_df = pd.concat([hiner_train_df, naamapadam_train_df], ignore_index=True)

In [6]:
filtered_rows = []

for idx in range(len(combined_ner_df)):
    tag_count = len(combined_ner_df['ner_tags'][idx]) - (combined_ner_df['ner_tags'][idx].count(0) + combined_ner_df['ner_tags'][idx].count(22))
    if tag_count > 3:
        filtered_rows.append(combined_ner_df.iloc[idx])

clean_ner_df = pd.DataFrame(filtered_rows)
clean_ner_df.to_csv('clean_hindi_ner.csv', index=False)

In [11]:
from transformers import BertTokenizerFast, BertForTokenClassification,DataCollatorForTokenClassification

In [8]:
tokenizer_ner = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
model_ner = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=23)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
df = pd.read_csv("clean_hindi_ner.csv")
df['tokens'] = df['tokens'].apply(ast.literal_eval)
df['ner_tags'] = df['ner_tags'].apply(ast.literal_eval)

In [ ]:
from datasets import Dataset

def align_labels_with_tokens(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer_ner(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(align_labels_with_tokens, batched=True, batch_size=128)
data_collator=DataCollatorForTokenClassification(tokenizer_ner)


Map:   0%|          | 0/424148 [00:00<?, ? examples/s]

In [ ]:
device_ner = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs_ner = 100
batch_size_ner = 2
lr_ner = 2e-5
weight_decay_ner = 0.01

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    input_ids = [torch.tensor(item["input_ids"]) for item in batch]
    attention_mask = [torch.tensor(item["attention_mask"]) for item in batch]
    labels = [torch.tensor(item["labels"]) for item in batch]
    input_ids = pad_sequence(input_ids, batch_first=True)
    attention_mask = pad_sequence(attention_mask, batch_first=True)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

train_ner_loader = DataLoader(tokenized_dataset, batch_size=batch_size_ner, shuffle=True, collate_fn=custom_collate_fn)
optimizer_ner = torch.optim.AdamW(model_ner.parameters(), lr=lr_ner, weight_decay=weight_decay_ner)
model_ner.to(device_ner)


In [ ]:
model_ner.train()
for epoch in range(epochs_ner):
    total_loss_ner = 0
    for batch in train_ner_loader:
        input_ids = batch["input_ids"].to(device_ner)
        attention_mask = batch["attention_mask"].to(device_ner)
        labels = batch["labels"].to(device_ner)

        optimizer_ner.zero_grad()

        outputs = model_ner(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer_ner.step()

        total_loss_ner += loss.item()

    avg_loss_ner = total_loss_ner / len(train_ner_loader)
    print(f"Epoch {epoch+1}/{epochs_ner}, Loss: {avg_loss_ner}")


In [ ]:
test_split_ratio = 0.2
train_df_ner, test_df_ner = train_test_split(df, test_size=test_split_ratio, random_state=42)

print("Test set shape:", test_df_ner.shape)

test_df_ner.to_json("/path/to/save/test_ner_data.json", orient="records", lines=True)

test_ner_dataset = pd.read_json('/path/to/save/test_ner_data.json', lines=True)

test_ner_dataset = Dataset.from_pandas(test_ner_dataset)


In [ ]:
test_tokenized_ner = test_ner_dataset.map(align_labels_with_tokens, batched=True, batch_size=128)

predictions = []
for sample in test_tokenized_ner:
    text = " ".join(sample['tokens'])
    predicted_entities = ner_classifier(text)
    temp = []
    for entity in predicted_entities:
        temp.append(entity['entity'])
    predictions.append(temp)


In [ ]:
references_ner = []
for sample in test_tokenized_ner:
    labels = sample['labels']
    temp = []
    for idx in range(1, len(labels)-1):
        temp.append('LABEL_' + str(labels[idx]))
    references_ner.append(temp)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def calculate_metrics(true_labels, pred_labels):
    metrics = {
        "precision": precision_score(true_labels, pred_labels, average='macro'),
        "recall": recall_score(true_labels, pred_labels, average='macro'),
        "f1": f1_score(true_labels, pred_labels, average='macro'),
        "accuracy": accuracy_score(true_labels, pred_labels)
    }
    return metrics

calculate_metrics(predictions, references_ner)


In [ ]:
!pip install fastapi uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI()

model_path = 'path/to/your/model'
tokenizer_ner = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
model_ner = BertForTokenClassification.from_pretrained(model_path)
ner_pipeline = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner)

class TextInput(BaseModel):
    text: str

@app.post("/ner")
def get_ner(input: TextInput):
    ner_results = ner_pipeline(input.text)
    return
